### Model implementation

In [20]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import scipy.stats as st

Import a test dataset

In [8]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

Split Dataset

In [9]:
# 80/20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Logistic Regression

Choose Solver

In [10]:
# newton-cg, sag where used
clf = LogisticRegression(random_state=0, solver='newton-cg').fit(X_train, y_train)
clf.predict(X_test)
clf.score(X_test, y_test)

1.0

## Choquistic Regression

In [11]:
class ChoquisticRegression(LogisticRegression):
    def __init__(self, additive='full', gamma=1.0, solver='lbfgs', max_iter=100, tol=1e-4):
        """
        additive: 'full' for unrestricted game-based aggregation, or '2-additive'
                  for the simplified variant.
        gamma: scaling factor (see Equations (20) and (24) in the paper).
        """
        self.additive = additive
        self.gamma = gamma
        super().__init__(solver=solver, max_iter=max_iter, tol=tol)
    
    def decision_function(self, X):
        f_val = super().decision_function(X)
        return self.gamma * f_val  # Placeholder for game-theoretic aggregation
    
    def predict_proba(self, X):
        scores = self.decision_function(X)
        proba = 1 / (1 + np.exp(-scores))
        return np.vstack([1 - proba, proba]).T

## ML Regression

In [19]:
class MultilinearLogisticRegression(LogisticRegression):
    def __init__(self, additive='full', gamma=1.0, solver='lbfgs', max_iter=100, tol=1e-4):
        """
        additive: 'full' for unrestricted game-based aggregation, or '2-additive'
                  for the simplified variant.
        gamma: scaling factor (see Equations (20) and (24) in the paper).
        """
        self.additive = additive
        self.gamma = gamma
        super().__init__(solver=solver, max_iter=max_iter, tol=tol)
    
    def decision_function(self, X):
        f_val = super().decision_function(X)
        return self.gamma * f_val  # Placeholder for game-theoretic aggregation
    
    def predict_proba(self, X):
        scores = self.decision_function(X)
        proba = 1 / (1 + np.exp(-scores))
        return np.vstack([1 - proba, proba]).T

## Helping functions

In [17]:
def compute_confidence_interval(accuracies, confidence=0.95):
    """
    Compute the mean accuracy and its 95% confidence interval.
    """
    mean_acc = np.mean(accuracies)
    sem = st.sem(accuracies)
    margin = sem * st.t.ppf((1 + confidence) / 2, len(accuracies) - 1)
    return mean_acc, (mean_acc - margin, mean_acc + margin)

def simulate_training_accuracy(model, X, y, n_simulations=50, test_size=0.2, random_state=42):
    """
    Run multiple training simulations (splitting data into train/test)
    and return the mean training accuracy (in %) with its 95% confidence interval.
    """
    accuracies = []
    for i in range(n_simulations):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, stratify=y, random_state=random_state + i
        )
        model.fit(X_train, y_train)
        acc = model.score(X_train, y_train)
        accuracies.append(acc * 100)  # Convert to percentage
    return compute_confidence_interval(accuracies)


## Results

In [18]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X, y = data.data, data.target

# Define models:
models = {
    "LR": LogisticRegression(solver='newton-cg', max_iter=100, tol=1e-4),
    "CR2add": ChoquisticRegression(additive='2-additive', solver='newton-cg', max_iter=100, tol=1e-4),
    "MLR2add": MultilinearLogisticRegression(additive='2-additive', solver='newton-cg', max_iter=100, tol=1e-4)
}

results = {}
for name, model in models.items():
    mean_acc, conf_interval = simulate_training_accuracy(model, X, y, n_simulations=50)
    results[name] = (mean_acc, conf_interval)
    print(f"{name}: {mean_acc:.2f}% {conf_interval}")

LR: 96.05% (np.float64(95.8752001654116), np.float64(96.23029434008296))
CR2add: 96.05% (np.float64(95.8752001654116), np.float64(96.23029434008296))
MLR2add: 96.05% (np.float64(95.8752001654116), np.float64(96.23029434008296))
